In [139]:
# Import neccessary libaries

import pandas as pd # Dataframes
import numpy as np # Arrays and Math

# Visual libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Warnings set to Ignore messages
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Import the sample dataset as a DataFrame
data = pd.read_csv('ted_main.csv')

In [11]:
# Create a copy DataFrame, keeping data as backup if needed
df = data.copy()

In [12]:
# View the top 5 rows of data
df.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869


In [13]:
# View the bottom 5 rows of data
df.tail()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
2545,17,"Between 2008 and 2016, the United States depor...",476,TED2017,1496707200,4,Duarte Geraldino,Duarte Geraldino: What we're missing in the de...,1,1505851216,"[{'id': 3, 'name': 'Courageous', 'count': 24},...","[{'id': 2596, 'hero': 'https://pe.tedcdn.com/i...",Journalist,"['TED Residency', 'United States', 'community'...",What we're missing in the debate about immigra...,https://www.ted.com/talks/duarte_geraldino_wha...,450430
2546,6,How can you study Mars without a spaceship? He...,290,TED2017,1492992000,3,Armando Azua-Bustos,Armando Azua-Bustos: The most Martian place on...,1,1505919737,"[{'id': 22, 'name': 'Fascinating', 'count': 32...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Astrobiologist,"['Mars', 'South America', 'TED Fellows', 'astr...",The most Martian place on Earth,https://www.ted.com/talks/armando_azua_bustos_...,417470
2547,10,Science fiction visions of the future show us ...,651,TED2017,1492992000,1,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,1,1506006095,"[{'id': 1, 'name': 'Beautiful', 'count': 14}, ...","[{'id': 2346, 'hero': 'https://pe.tedcdn.com/i...",Robotics engineer,"['AI', 'ants', 'fish', 'future', 'innovation',...",What intelligent machines can learn from a sch...,https://www.ted.com/talks/radhika_nagpal_what_...,375647
2548,32,In an unmissable talk about race and politics ...,1100,TEDxMileHigh,1499472000,1,Theo E.J. Wilson,Theo E.J. Wilson: A black man goes undercover ...,1,1506024042,"[{'id': 11, 'name': 'Longwinded', 'count': 3},...","[{'id': 2512, 'hero': 'https://pe.tedcdn.com/i...",Public intellectual,"['Internet', 'TEDx', 'United States', 'communi...",A black man goes undercover in the alt-right,https://www.ted.com/talks/theo_e_j_wilson_a_bl...,419309
2549,8,With more than half of the world population li...,519,TED2017,1492992000,1,Karoliina Korppoo,Karoliina Korppoo: How a video game might help...,1,1506092422,"[{'id': 21, 'name': 'Unconvincing', 'count': 2...","[{'id': 2682, 'hero': 'https://pe.tedcdn.com/i...",Game designer,"['cities', 'design', 'future', 'infrastructure...",How a video game might help us build better ci...,https://www.ted.com/talks/karoliina_korppoo_ho...,391721


In [14]:
# Inspect the overall shape of the data (rows/columns)
df.shape

(2550, 17)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2550 non-null   int64 
 1   description         2550 non-null   object
 2   duration            2550 non-null   int64 
 3   event               2550 non-null   object
 4   film_date           2550 non-null   int64 
 5   languages           2550 non-null   int64 
 6   main_speaker        2550 non-null   object
 7   name                2550 non-null   object
 8   num_speaker         2550 non-null   int64 
 9   published_date      2550 non-null   int64 
 10  ratings             2550 non-null   object
 11  related_talks       2550 non-null   object
 12  speaker_occupation  2544 non-null   object
 13  tags                2550 non-null   object
 14  title               2550 non-null   object
 15  url                 2550 non-null   object
 16  views               2550

The dataset is comprised of 2,550 rows and 17 columns.  There are **no null values** out of the 2,550 entries, however, Speaker Occupation appears to have 4 **duplicate values** which could be NAs and will require further analysis.

In [33]:
# Inspect total count of null values in dataset
df.isnull().sum()

comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    6
tags                  0
title                 0
url                   0
views                 0
dtype: int64

In [35]:
# Inspect total count of NA (NaN) values in dataset
df.isna().sum()

comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    6
tags                  0
title                 0
url                   0
views                 0
dtype: int64

Due to the fact that there are **only 6 speakers with missing occupations**, all of whom can likely be searched online for generic occupations, these entries can be **manually encoded**.

In [150]:
# Create a list of speakers with missing occupations
speakers = df['main_speaker'][df['speaker_occupation'].isna() == True].values.tolist()

# create an, ordered, input list for respective speaker occupations (from online search)
occupations_input = ['Designer', 'Entrepreneur', 'CEO', 'Inventor', 'Artist', 'Journalist']

# for loop to update each speaker with respective occupation that was missig
for speaker in speakers:
    for occupation in occupations_input:
        df['speaker_occupation'][df['main_speaker'] == speaker] = occupation

In [153]:
# Recheck the count of NAs to ensure above loop removed all
df.isna().sum()

comments              0
description           0
duration              0
event                 0
film_date             0
languages             0
main_speaker          0
name                  0
num_speaker           0
published_date        0
ratings               0
related_talks         0
speaker_occupation    0
tags                  0
title                 0
url                   0
views                 0
dtype: int64

In [154]:
# Run descriptive statistics summary for all variables - including categorical
df.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
comments,2550.0,NaN,NaN,NaN,191.562353,282.315223,2.0,63.0,118.0,221.75,6404.0
description,2550,2550,Sir Ken Robinson makes an entertaining and pro...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
duration,2550.0,NaN,NaN,NaN,826.510196,374.009138,135.0,577.0,848.0,1046.75,5256.0
event,2550,355,TED2014,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN
film_date,2550.0,NaN,NaN,NaN,1321927552.047059,119739059.699232,74649600.0,1257465600.0,1333238400.0,1412964000.0,1503792000.0
languages,2550.0,NaN,NaN,NaN,27.326275,9.563452,0.0,23.0,28.0,33.0,72.0
main_speaker,2550,2156,Hans Rosling,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,2550,2550,Ken Robinson: Do schools kill creativity?,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
num_speaker,2550.0,NaN,NaN,NaN,1.028235,0.207705,1.0,1.0,1.0,1.0,5.0
published_date,2550.0,NaN,NaN,NaN,1343525265.750588,94640090.798582,1151367060.0,1268462970.0,1340935446.5,1423432130.25,1506092422.0


**Initial Sample Observations**
- The most frequest event (year) is TED2014 (84 occurences)
- Hans Rosling was the most frequent speaker (9 presentations)
- The most common speaker occupation is Writer (45 occurences)

In [23]:
# Inspect the top 10 speaker occupations
df['speaker_occupation'].value_counts().head(10)

Writer          45
Artist          34
Designer        34
Journalist      33
Entrepreneur    31
Architect       30
Inventor        27
Psychologist    26
Photographer    25
Filmmaker       21
Name: speaker_occupation, dtype: int64